In [1]:
import json
import pandas as pd
import glob
import xmltodict, json

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
files = glob.glob('IADownloads/*')

rows = []

for file in files:
    row = []
    with open(file + '/' + file.split('/')[1] + '_meta.xml') as f:
        
        try:
            jsn = xmltodict.parse(f.read())
            
            row.append(jsn['metadata']['identifier'])
            row.append(jsn['metadata']['collection'][1])
            row.append(jsn['metadata']['mediatype'])
            row.append(jsn['metadata']['title'])
            if 'date' in jsn['metadata']:
                row.append(jsn['metadata']['date'])
            elif 'year' in jsn['metadata']:
                row.append(jsn['metadata']['year'])
            else:
                row.append('')
            if 'description' in jsn['metadata']:
                row.append(jsn['metadata']['description'])
            else:
                row.append('')
            row.append(','.join(jsn['metadata']['subject'].split(';')[:-1]))
        except:
            print('error in', file)
    rows.append(row)

In [4]:
df_metadata = pd.DataFrame(rows, columns=['identifier', 'collection', 'title', 'mediatype', 'year', 'description','subject'])

In [5]:
df_metadata

,identifier,collection,title,mediatype,year,description,subject
0,lnq23e00,newsandpublicaffairs,movies,Seizing the Initiative: Action on Environmental Tobacco Smoke,1991,"INFOTAB, a tobacco industry information clearinghouse, offers an informational program for national tobacco manufacturers to develop a strategy to address the environmental tobacco smoke issue.","cigarette , international trade , anti-smoking advocacy , secondhand smoke , industry strategy , marketing research , political activity , public relations , workplace , mass transit , airplane"
1,tobacco_two47c00,newsandpublicaffairs,movies,Master Ventilation Innovation R and D,,Video that compares a traditional HVAC system with a new displacement ventilation system. The latter system filters out particulate matter and nicotine and air enters a room from the floor and exits through the ceiling which minimizes mixing.,"tobacco , cigarette , secondhand smoke , indoor air quality , tobacco smoke"
2,tobacco_wvp23e00,newsandpublicaffairs,movies,PM USA Video Newsletter 1987/06/00,1987,"Fourth issue of the video newsletter, aimed at the Philip Morris sales force, includes segments on Vons Pavillions, entertainment and sporting events sponsored by Philip Morris, the Food Marketing Institute's annual convention, and a report on the financial health of the company.","tobacco , cigarette , sales , sports sponsorship , event sponsorship , retail outlet, marketing , female , promotions , alcoholic beverage , food, business activity"
3,tobacco_sdo23e00,newsandpublicaffairs,movies,Philip Morris 'I Love Lucy',1964,Animated and live commercials with Lucille Ball and Desi Arnaz advertising Philip Morris cigarettes.,"tobacco , cigarette , advertising, television"
4,opioids_tqph0257,newsandpublicaffairs,movies,"A change is coming, at effervescent speed : Fentora (fentanyl buccal tablet)",2006,"<span style=""color:rgb(51,51,51);font-family:'Helvetica Neue', Helvetica, Arial, sans-serif;font-size:12.1884px;background-color:rgb(255,255,255);"">Consists of a Fentora timeline, media coverage, statistics, etc.; Created in anticipation of full approval of Fentora, and/or Phase IV trials of Fentora; Contains the entirety of the song “Speed of Sound” [Radio Edit] by Coldplay, divided into 4 segments (0:10 to 2:14, 2:47 to 3:18, 3:42 to 3:59, and 4:41 to 6:10).</span>","opioids, sales and marketing, fentora, pharmaceutical industry"


In [6]:
files = glob.glob('IADownloads/*')

rows = []
for file in files:
    with open(file + '/' + file.split('/')[1] + '_files.xml') as f:
        try:
            jsn = xmltodict.parse(f.read())
            for f in jsn['files']['file']:  
                #print(f)
                #print('***', f['@name'].split('.')[-1])
                row = []
                if f['@name'].split('.')[-1] == 'mp4':  
                    dl = f['@name']
                    s = int(f['size'])
                    url = 'https://archive.org/download/' + file.split('/')[1] + '/' + dl
                    row.append(file.split('/')[1])
                    row.append(dl)
                    row.append(s)
                    row.append(url)
                    #print(row)
                    rows.append(row)
        except:
            print('error in', file)

In [7]:
df_files = pd.DataFrame(rows, columns=['identifier', 'name', 'size', 'url'])

In [8]:
df_files

,identifier,name,size,url
0,lnq23e00,VTS_01_1_512kb.mp4,43861974,https://archive.org/download/lnq23e00/VTS_01_1_512kb.mp4
1,tobacco_two47c00,VTS_01_1_512kb.mp4,66659846,https://archive.org/download/tobacco_two47c00/VTS_01_1_512kb.mp4
2,tobacco_wvp23e00,VTS_01_1_512kb.mp4,123014161,https://archive.org/download/tobacco_wvp23e00/VTS_01_1_512kb.mp4
3,tobacco_sdo23e00,10001665-0-2058500033_chunk_1_512kb.mp4,5887037,https://archive.org/download/tobacco_sdo23e00/10001665-0-2058500033_chunk_1_512kb.mp4
4,opioids_tqph0257,opioids_tqph0257.mp4,36771097,https://archive.org/download/opioids_tqph0257/opioids_tqph0257.mp4


In [9]:
df = pd.merge(df_files, df_metadata,on='identifier')

In [10]:
df.to_csv('data/links_with_metadata.csv', index=False)

In [11]:
df

,identifier,name,size,url,collection,title,mediatype,year,description,subject
0,lnq23e00,VTS_01_1_512kb.mp4,43861974,https://archive.org/download/lnq23e00/VTS_01_1_512kb.mp4,newsandpublicaffairs,movies,Seizing the Initiative: Action on Environmental Tobacco Smoke,1991,"INFOTAB, a tobacco industry information clearinghouse, offers an informational program for national tobacco manufacturers to develop a strategy to address the environmental tobacco smoke issue.","cigarette , international trade , anti-smoking advocacy , secondhand smoke , industry strategy , marketing research , political activity , public relations , workplace , mass transit , airplane"
1,tobacco_two47c00,VTS_01_1_512kb.mp4,66659846,https://archive.org/download/tobacco_two47c00/VTS_01_1_512kb.mp4,newsandpublicaffairs,movies,Master Ventilation Innovation R and D,,Video that compares a traditional HVAC system with a new displacement ventilation system. The latter system filters out particulate matter and nicotine and air enters a room from the floor and exits through the ceiling which minimizes mixing.,"tobacco , cigarette , secondhand smoke , indoor air quality , tobacco smoke"
2,tobacco_wvp23e00,VTS_01_1_512kb.mp4,123014161,https://archive.org/download/tobacco_wvp23e00/VTS_01_1_512kb.mp4,newsandpublicaffairs,movies,PM USA Video Newsletter 1987/06/00,1987,"Fourth issue of the video newsletter, aimed at the Philip Morris sales force, includes segments on Vons Pavillions, entertainment and sporting events sponsored by Philip Morris, the Food Marketing Institute's annual convention, and a report on the financial health of the company.","tobacco , cigarette , sales , sports sponsorship , event sponsorship , retail outlet, marketing , female , promotions , alcoholic beverage , food, business activity"
3,tobacco_sdo23e00,10001665-0-2058500033_chunk_1_512kb.mp4,5887037,https://archive.org/download/tobacco_sdo23e00/10001665-0-2058500033_chunk_1_512kb.mp4,newsandpublicaffairs,movies,Philip Morris 'I Love Lucy',1964,Animated and live commercials with Lucille Ball and Desi Arnaz advertising Philip Morris cigarettes.,"tobacco , cigarette , advertising, television"
4,opioids_tqph0257,opioids_tqph0257.mp4,36771097,https://archive.org/download/opioids_tqph0257/opioids_tqph0257.mp4,newsandpublicaffairs,movies,"A change is coming, at effervescent speed : Fentora (fentanyl buccal tablet)",2006,"<span style=""color:rgb(51,51,51);font-family:'Helvetica Neue', Helvetica, Arial, sans-serif;font-size:12.1884px;background-color:rgb(255,255,255);"">Consists of a Fentora timeline, media coverage, statistics, etc.; Created in anticipation of full approval of Fentora, and/or Phase IV trials of Fentora; Contains the entirety of the song “Speed of Sound” [Radio Edit] by Coldplay, divided into 4 segments (0:10 to 2:14, 2:47 to 3:18, 3:42 to 3:59, and 4:41 to 6:10).</span>","opioids, sales and marketing, fentora, pharmaceutical industry"
